In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
train
# 집값을 예측하는 대회
# 수치형 범주들이 많음!

In [ ]:
# 아래 산점도에서 본 outlier제거
train[(train['GrLivArea'] > 4500) & (train['SalePrice'] < 300000)]

In [ ]:
# 위 두 컬럼 제거
drop_index = train[(train['GrLivArea'] > 4500) & (train['SalePrice'] < 300000)].index
drop_index

In [ ]:
train = train.drop(drop_index)
train

In [ ]:
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
test

In [ ]:
train.corr()['SalePrice'].sort_values(ascending=False)
# y값(SalePrice)에 x값이 영향을 미치는 순으로 추출!
# 타 대회들에 비해서 x들의 y에 영향을 미치는 점수가 높은편!
# 수치형 변수가 많으므로 catboost를 쓴다고 점수가 높게 잘 안나온다!
# 다른모델(수치형 변수들 모델링 잘하는)으로 바꿔줄것! --> 선형모델

In [ ]:
# 산점도(scatterplot)를 통한 이상치 파악
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,8))
sns.scatterplot(train['GrLivArea'],train['SalePrice'])

In [ ]:
pd.options.display.max_columns=999

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
# 파생변수 생성(집이 얼마나 오래됐는지 --> Old)
all_data['Old'] = all_data['YrSold'] - all_data['YearBuilt']

In [ ]:
# 도메인 지식이 없을때 파생변수 만드는 tip
# 1) y값에 영향많이 주는 컬럼들을 잘 조합해 파생변수를 만들면 점수up!
# 2) corr 계수가 낮은 변수여도 파생변수 조합을 잘하면
#   새로운 방향성 설정을 통해 y값최적해의 탐색에 도움을 줌!

In [ ]:
# 파생변수 생성2
# OverallQual + OverallCond --> 집이 품질도 좋고 상태도 좋다!
all_data['Overall_plus'] = all_data['OverallQual'] + all_data['OverallCond']

In [ ]:
# 추가
#OverallQual - OverallCond : 집 품질과 상태의 차이를 나타내는 파생변수 생성
all_data['Overall_minus'] = all_data['OverallQual'] - all_data['OverallCond']

In [ ]:
# 파생변수 생성3
# 집 내부의 총 면적
all_data['TotalArea'] = all_data['GrLivArea'] + all_data['TotalBsmtSF']

In [ ]:
# 파생변수 생성4
# 집 내부 면적 + 집 외부 면적
all_data['FinalArea'] = all_data['TotalArea'] + all_data['GarageArea']

In [ ]:
# GarageArea는 되는데 LotArea는 안되는 이유? 
# --> LotArea는 컬럼의 중요도도 낮은데 데이터의 범위도 크므로
# FinalArea에 추가하면 의미가 없다
all_data[['LotArea','GarageArea']]

In [ ]:
# 가중치를 통해서 LotArea의 영향력을 줄인다!
# 이후 FinalArea에 추가하면 어느정도 점수향상에 보탬이 됨!
all_data['area'] = all_data['FinalArea'] + (all_data['LotArea'] * 0.1)

In [ ]:
all_data_2 = all_data.drop(columns = ['Id','SalePrice'])
all_data_2

In [ ]:
# 선형모델에서 범주형인데 숫자로 인식하는 컬럼을 문자로 바꿔주었다(MSSubClass)
all_data_2['MSSubClass'] = all_data_2['MSSubClass'].astype('str')

In [ ]:
# 이 대회에서는 labelencoding 사용 X
# 사용할 선형모델은 수치형 컬럼은 잘 해석하나 범주형 컬럼은 아님
# 범주형 컬럼 --> one-hot-encoding 처리
all_data_3 = pd.get_dummies(all_data_2)
all_data_3

In [ ]:
# 결측치 처리
all_data_3 = all_data_3.fillna(-1)

In [ ]:
# Tree모델일때는 각 컬럼들이 y값에 독립적인 영향을 줬는데
# 선형모델은 y값 예측시 y = ax1 + bx2 + cx3 ... 이런식으로 복합적인 영향을 주므로
# 컬럼간 값의 범위를 맞춘다(Scaling)
# standard scaler --> 평균을 0, 분산을 1로 조정함
'''
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
all_data_4 = ss.fit_transform(all_data_3)
'''
# Standard Scaler 장점 : 사용이 쉬움
# 단점 : outlier가 많으면 해석을 잘 못함
#      데이터 양이 적으면 해석을 잘 못함(outlier에 영향 쉽게 받으므로)

In [ ]:
# Robust Scaler : 중앙값을 가지고 scaling --> outlier 있어도 영향 거의 없음
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
all_data_4 = rs.fit_transform(all_data_3)
all_data_4
# 익숙하지 않은 데이터형... --> array 형식
# --> 데이터 프레임으로 변경

In [ ]:
all_data_4 = pd.DataFrame(all_data_4,columns=all_data_3.columns)
all_data_4

In [ ]:
train_2 = all_data_4[:len(train)]
test_2 = all_data_4[len(train):]

In [ ]:
# linear regression 모델 : 옵션을 못줌 --> 모든 컬럼을 동일 가중치로 학습한다!
#                     --> 주제를 포함가능한 선형모델(Ridge)를 사용
from sklearn.linear_model import Ridge
rg = Ridge(alpha=21)
rg.fit(train_2,np.log(train['SalePrice']))
result = rg.predict(test_2)
result

In [ ]:
from catboost import CatBoostRegressor
cb = CatBoostRegressor(verbose=200)
cb.fit(train_2,np.log(train['SalePrice']))
result_cb = cb.predict(test_2)
result_cb

In [ ]:
sub = pd.read_csv('/kaggle/input/home-data-for-ml-course/sample_submission.csv')
sub

In [ ]:
sub['SalePrice'] = np.exp(result_cb) * 0.5 + np.exp(result) * 0.5
# 지수함수 처리 --> 로그를 원본데이터로 변환
# ensemble을 위해 각 result에 동일 가중치(0.5) 부여
sub.to_csv('result.csv',index=False)

In [ ]:
sub['SalePrice'] = np.exp(result_cb) * 0.5 + np.exp(result) * 0.5
# 지수함수 처리 --> 로그를 원본데이터로 변환
# ensemble을 위해 각 result에 동일 가중치(0.5) 부여
sub.to_csv('result.csv',index=False)

#### 선형모델 / Tree모델 점수 비슷비슷!
#### --> 서로 잘 해주지 못하는 부분들을 보완해주자!(Ensemble)

#### 앙상블 개선개념 --> 카톡참조!
#### 주의점 : 각 모델이 다른 종류일수록 좋음!  
#### --> 비슷한 종류의 모델을 섞을때는 가중치를 0.5말고 다른 비율로 섞는다!
#### 그것도 귀찮다면 그냥 아예 다른 종류의 모델 2개를 섞는게 낫다!         

#### 스태킹 : 각 모델의 예측값을 가져와 패턴발견 --> 자동으로 가중치를 부여함

In [ ]:
# 내부적으로 점수파악하기
# 데이터가 많을때는 validation 데이터를 20%만 써도 괜찮은데,
# 데이터가 적을때는 validation 데이터셋을 잘 뽑아야 한다!
# --> 교차검증 : 모든 데이터를 학습에 사용, 모든 train데이터셋을 valid데이터셋으로 이용가능!
# --> 편향된 시각 보완가능!
from sklearn.model_selection import cross_val_score
cvs = cross_val_score(rg,train_2,train['SalePrice'],n_jobs=-1,cv=10,
                     scoring='neg_mean_absolute_error')
# cross_val_score(모델, 독립변수, 종속변수, n_jobs=-1, cv=반복횟수, scoring='neg_mean_absolute')
# neg_mean_absolute : 우리모델의 점수예측방식 --> 사이킷런의 결함으로 
#                 앞에 neg를 붙이므로 결과를 볼때는 앞에 -를 꼭 붙여준다!
-cvs.mean()

# https://cinema4dr12.tistory.com/1275

In [ ]:
# 하이퍼파라미터 튜닝 : 우리가 쓸 모델의 가장 좋은 옵션은 무엇인가?
# for문 작성 후 alpha가 몇일때 점수가 제일 높은지 확인!
for i in [10,20,50,100,200,300,500,700,1000] :
    rg_cv = Ridge(alpha=i)
    cvs = cross_val_score(rg_cv,train_2,train['SalePrice'],n_jobs=-1,cv=10,
                         scoring='neg_mean_absolute_error')
    print(i, ":", -cvs.mean())

# Standard Scaling  : alpha = 300정도일때쯤 제일 좋은듯

# Robust Scaling을 사용시 --> alpha값을 적게 주는것이 더 효과가 좋은 것을 확인!

In [ ]:
for i in range(10,30) :
    rg_cv = Ridge(alpha=i)
    cvs = cross_val_score(rg_cv,train_2,train['SalePrice'],n_jobs=-1,cv=10,
                         scoring='neg_mean_absolute_error')
    print(i, ':', -cvs.mean())
# alpha = 21일때쯤이 최적!

In [ ]:
# 점수를 더 올리려면?
# --> 회귀문제를 잘 풀기 위해서는 y값의 분포를 확인해야 한다!
# 그 분포가 특이할 때는 로그함수를 취하면 점수 개선
import seaborn as sns
sns.distplot(train['SalePrice'])
# 분포모양 : 우측꼬리가 긴 모양(대부분의 y값 데이터는 10만~20만 사이에 몰림)
#         outlier들이 많이 존재
# 이 상태로 학습하면 적은 데이터부분을 잘 학습하지 못하고 학습범위도 넓어져 모델이 잘못된 방법으로 학습
# --> 로그를 취해주자!

In [ ]:
import seaborn as sns
sns.distplot(np.log(train['SalePrice']))
# 분포가 정규분포 모양이 되었음 --> 안정적인 학습
# 꼬리가 너무 길지 않아짐! --> outlier 학습 해결 + 학습하는 범위가 짧아짐!